In [6]:
#llm = LLM(model="EleutherAI/gpt-j-6b")

INFO 10-12 01:22:20 config.py:1652] Downcasting torch.float32 to torch.float16.
INFO 10-12 01:22:20 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='EleutherAI/gpt-j-6b', speculative_config=None, tokenizer='EleutherAI/gpt-j-6b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=EleutherAI/gpt-j-6b, use_v2_block_man

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/vllm/model_executor/model_loader/weight_utils.py:424: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any i

INFO 10-12 01:26:25 model_runner.py:1025] Loading model weights took 11.2714 GB


INFO 10-12 01:26:25 gpu_executor.py:122] # GPU blocks: 1363, # CPU blocks: 585
INFO 10-12 01:26:27 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-12 01:26:27 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-12 01:26:49 model_runner.py:1456] Graph capturing finished in 22 secs.


In [34]:
from vllm import LLM, SamplingParams

prompts = [
    "Make a sentence using the verb 'retire'.  Sentence: ",
    "Make a sentence using the verb 'retire'.  Sentence: ",
    "Make a sentence using the verb 'retire'.  Sentence: ",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens = 40)

In [35]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  5.03it/s, est. speed input: 75.41 toks/s, output: 201.08 toks/s]

Prompt: "Make a sentence using the verb 'retire'.  Sentence: ", Generated text: "\n\nI like to retire in the afternoon.\n\nYou'll notice you've used the verb 'retire' in the sentence.  The verb 'retire' is a transitive verb"
Prompt: "Make a sentence using the verb 'retire'.  Sentence: ", Generated text: '\n\nRetirement is ____, but it can be a pleasant change.\n\nA:\n\nThe word you are looking for is abdicate. \nIt has the meaning of'
Prompt: "Make a sentence using the verb 'retire'.  Sentence: ", Generated text: '\n\nBy the time you retire, you will be 60 years old.\n\nWhat I made: \n\nBy the time you retire, you will be 60 years old.\n\nA'


### Try out Llama-8b

In [6]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
PAD_TOKEN = "<|pad|>"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"
# takes 2-3 minutes
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    #     attn_implementation="flash_attention_2",
    #     attn_implementation="sdpa",
    device_map="auto",
)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]


Embedding(128264, 4096)

In [8]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)

In [13]:
# template
def create_test_prompt(word: str):
    prompt_format = "make three sentences in Japanese using the verb {}."
    prompt = prompt_format.format(word)
    messages = [
        {"role": "user", "content": prompt},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [16]:
# let's run some inference
# 1. generation task

# system warmup
SYSTEM_START = "You can speak both japanese and english fluently. Use Hiragana and kanji."
USER_START = "make three sentences in Japanese using the verb '慣れる'"
warmup_prompt = tokenizer.apply_chat_template(
        [{
            "role": "system",
            "content": SYSTEM_START,
        }, {
            "role": "user",
            "content": USER_START,
        }
        ], tokenize=False, add_generation_prompt=True )
outputs = pipe(warmup_prompt)

# start
WORDS_LIST = ["𠮟る","変える","行なう", "冷める"]
prompts = [create_test_prompt(word) for word in WORDS_LIST]
for prompt in prompts:
    print(prompt)
    print("\nPrediction:")
    outputs = pipe(prompt)
    print(outputs[0]["generated_text"])
    print("\n\n")

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

make three sentences in Japanese using the verb 𠮟る.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Prediction:
Here are three sentences in Japanese using the verb 𠮟る (miru, meaning "to see" or "to look"):

1. (Watashi wa ki no ji o miru.) - I look at the tree's branches.
2. (Kare wa sono inu o miru.) - He looks at that dog.
3. (Watashi wa asa made no sore o miru.) - I look at the sunrise until morning.

Note: The verb 𠮟る (miru) can have different nuances depending on the context and the particles used. In these sentences, I used the



<|begin_of_text|><|start_header_id|>user<|end_header_id|>

make three sentences in Japanese using the verb 変える.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



Prediction:
Here are three sentences in Japanese using the verb 変える (kaeru), which means "to change" or "to exchange":

1. (Watashi wa kimono o kaeru.) - I will change my kimono.
2. (Kono inu wa neko o kaeru.) - This d

That was bullshit. Let's try a model that can speak Japanese.
sadly, there's only 70b model for llama 3 in japanese.
### Try out Llama2-Japanese-7b

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"
text = "クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"
# will take 4-6 minutes
model_name = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")


/opt/conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.30it/s]


In [3]:
print(f"BOS: {tokenizer.bos_token} -> {tokenizer.bos_token_id}")
print(f"EOS: {tokenizer.eos_token} -> {tokenizer.eos_token_id}")
print(f"PAD: {tokenizer.pad_token} -> {tokenizer.pad_token_id}")

BOS: <s> -> 1
BOS: </s> -> 2
PAD: </s> -> 2


In [18]:

SYSTEM_START = "あなたは誠実で優秀な日本人のアシスタントです。"
warmup_prompt = "{bos_token}{b_inst} {system}{e_inst} ".format(
    bos_token=tokenizer.bos_token,
    b_inst=B_INST,
    system=f"{B_SYS}{SYSTEM_START}{E_SYS}",
    e_inst=E_INST,
)
print(warmup_prompt)

def create_test_prompt(word: str):
    prompt_format = "{bos_token}{b_inst}make three different sentences in Japanese using the word '{word}'. {e_inst} "
    return prompt_format.format(
        bos_token=tokenizer.bos_token,
        b_inst=B_INST,
        word=word,
        e_inst=E_INST,
    )


WORDS_LIST = ["起きる","変える","行なう", "冷める", "謝る", "抱く", "決める", "高める"]
prompts = [create_test_prompt(word) for word in WORDS_LIST]
for prompt in prompts:
    print(prompt)

<s>[INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

[/INST] 
<s>[INST]make three different sentences in Japanese using the word '起きる'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '変える'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '行なう'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '冷める'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '謝る'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '抱く'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '決める'. [/INST] 
<s>[INST]make three different sentences in Japanese using the word '高める'. [/INST] 


In [8]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    return_full_text=False,
)


In [14]:
import time
system_output = pipe(warmup_prompt)
for prompt in prompts:
    print(prompt)
    print("\nPrediction:")
    outputs = pipe(prompt)
    print(outputs[0]["generated_text"])
    print("\n\n")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


<s>[INST]make three different sentences in Japanese using the word '起きる' in various forms or tense. [/INST] 

Prediction:
 承知しました。「起きる」の異なる形や時制で使われる例を3つお教えします: 

1. 明日、早起きしていい気分になりそうだ。
2. 今日は早く起きるぞ！
3. 毎朝、学校に行くのは苦痛で起きるのがつらい。

いかがでしょうか



<s>[INST]make three different sentences in Japanese using the word '変える' in various forms or tense. [/INST] 

Prediction:
 承知しました。「変える」の異なる形や時制での使い方を3つご紹介します。

1. 明日の予定を変える
    - 明日の予定を変える (変えるの動)
    - 明日の予定を変えよう (変えるの助)
    - 明日の予定を変えてみよう (変えるの助)



<s>[INST]make three different sentences in Japanese using the word '行なう' in various forms or tense. [/INST] 

Prediction:
 承知しました。「行なう」の異なる形や時制での使い方を3つご紹介します。

1. 今日は友達と一緒に映画を見る予定です。
    (友達と一緒に映画を見る予定で今日はいます。)
2. 明日は掃除をしないと行けないだろう。




<s>[INST]make three different sentences in Japanese using the word '冷める' in various forms or tense. [/INST] 

Prediction:
 承知しました。「冷める」を用いた3つの異なる文章を日本語で作成します。

1. コーヒーが冷める前に飲み切ってしまいました。
2. 彼女の態度が冷めてしまったのは、彼のアダルトサイトの存在が原因です。
3. この前の�





In [16]:
# measure inference time
import time

system_output = pipe(warmup_prompt)
duration = 0.0
NUM_WARMUP = 5
NUM_REPEATS = 20
for i in range(NUM_REPEATS+NUM_WARMUP):
    start_time = time.time()
    for prompt in prompts:
        outputs = pipe(prompt)
    if i > NUM_WARMUP:
        duration += time.time() - start_time
print(duration/NUM_REPEATS)

9.347641944885254


In [17]:
system_output = pipe(warmup_prompt)
duration = 0.0
for i in range(NUM_REPEATS+NUM_WARMUP):
    start_time = time.time()
    outputs = pipe(prompts, batch_size = len(prompts))
    if i > NUM_WARMUP:
        duration += time.time() - start_time
print(duration/NUM_REPEATS)

6.051364421844482


In [6]:
# let's look at the tokens more closely.
with torch.no_grad():
    token_ids = tokenizer.encode(prompts[2], add_special_tokens=False, return_tensors="pt")

    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

output_ids = output_ids.tolist()[0]
# individual tokens are just one kanji/hiragana character. not vocabulary
output = tokenizer.decode(output_ids[token_ids.size(1) :], skip_special_tokens=True)
print(output)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


承知しました。「行なう」を使った3つの日本語の文章は以下の通りです。

1. 彼女は毎日のように洗濯物を行なう。
2. 彼は毎週末に庭の草むしりを行なう。
3. 私は週に3回、ジムに行き走りを行なう。


In [51]:
import MeCab
from collections import namedtuple
tagger = MeCab.Tagger("--node-format=%f%m")
tagged = tagger.parse("彼女は毎日のように濯物洗を行なう").split('\n')

TaggedWord = namedtuple('TaggedWord', 'word, yomi, pos')

parts = []
for line in tagged:
    tags = line.split('\t')
    if len(tags) >= 5:
        parts.append(TaggedWord(word=tags[0], yomi=tags[1], pos=tags[4]))
    
print(parts)


[TaggedWord(word='彼女', yomi='カノジョ', pos='代名詞'), TaggedWord(word='は', yomi='ワ', pos='助詞-係助詞'), TaggedWord(word='毎日', yomi='マイニチ', pos='名詞-普通名詞-副詞可能'), TaggedWord(word='の', yomi='ノ', pos='助詞-格助詞'), TaggedWord(word='よう', yomi='ヨー', pos='形状詞-助動詞語幹'), TaggedWord(word='に', yomi='ニ', pos='助動詞'), TaggedWord(word='洗濯', yomi='センタク', pos='名詞-普通名詞-サ変可能'), TaggedWord(word='物', yomi='ブツ', pos='接尾辞-名詞的-一般'), TaggedWord(word='を', yomi='オ', pos='助詞-格助詞'), TaggedWord(word='行なう', yomi='オコナウ', pos='動詞-一般')]


In [34]:
import re

sents = re.split("\n[0-9][. ]*", "works out as follows\n1.  fast.\n2 slow 3. without newline!\n\n4. with new\n\n")
sents = [sent.strip() for sent in sents[1:]]
print(sents)

['fast.', 'slow 3. without newline!', 'with new']
